In [74]:
# importando a biblioteca para fazer operações em banco de dados sqlite3
import sqlite3
# importando a bibliotecas pandas para transformar dados e tabelas do banco em data frames 
import pandas as pd

In [76]:
# criaremos a conexão com nosso banco de dados
# se ele não existir, será criado
conn = sqlite3.connect("database.db")

In [78]:
# Podemos agora criar tabelas
# criaremos a tabela (table) tasks com campos (fields) id, description e done
conn.execute("""
    CREATE TABLE IF NOT EXISTS tasks
    (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        description TEXT,
        done INTEGER
    )    
""")
# após executar uma operação é necessário salvá-la
conn.commit()

In [80]:
# criaremos agora um registro (row) na tabela tasks
# o registro terá valor 'Estudar seaborn' para description e 0 para done.
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Estudar seaborn', 0);    
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

In [82]:
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Estudar python', 1)
""")

# Salvar as alterações
conn.commit()


In [84]:
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Entregar livro', 0)
""")

# Salvar as alterações
conn.commit()


In [86]:
# podemos agora visualizar todos (*) os registros da (FROM) tabela tasks
results = conn.execute("""
    SELECT *
    FROM tasks
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

(2, 'Estudar python', 1)
(3, 'Entregar livro', 0)
(4, 'Estudar seaborn', 0)
(5, 'Estudar python', 1)
(6, 'Entregar livro', 0)


In [88]:
# ver os valores de description da (FROM) tabela tasks onde (WHERE) done = 0
results = conn.execute("""
    SELECT description
    FROM tasks
    WHERE done=0
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

('Entregar livro',)
('Estudar seaborn',)
('Entregar livro',)


In [90]:
results = conn.execute("""
    SELECT description
    FROM tasks
    WHERE done = 0 AND description LIKE 'Estudar%'
""").fetchall()

# imprimindo cada resultado
for result in results:
    print(result)


('Estudar seaborn',)


In [92]:
# Atualizar (update) na tabela tasks as linhas (rows) cujo id = 1 fazendo (SET) done = 1
conn.execute("""
    UPDATE tasks
    SET done=1
    WHERE id=1
""")
# salvando as atualizações
conn.commit()

In [94]:
conn.execute("""
    UPDATE tasks
    SET description = 'Estudar matplotlib'
    WHERE description = 'Estudar seaborn'
""")

# salvando as atualizações
conn.commit()


In [96]:
# deletar da (FROM) tabela tasks as linhas (rows) onde id = 1
conn.execute("""
    DELETE FROM tasks
    WHERE id=1
""")
conn.commit()

In [98]:
df = pd.read_sql_query(
    """
       SELECT *
       FROM tasks
    """,
    conn
)
df

,id,description,done
0,2,Estudar python,1
1,3,Entregar livro,0
2,4,Estudar matplotlib,0
3,5,Estudar python,1
4,6,Entregar livro,0


In [100]:
df['done'].value_counts()

done
0    3
1    2
Name: count, dtype: int64

In [102]:
results = conn.execute("""
    SELECT done, COUNT(*) AS done_tasks
    FROM tasks
    GROUP BY done;      
""").fetchall()

for result in results:
    print(result)
conn.close()

(0, 3)
(1, 2)


In [104]:
conn = sqlite3.connect("database_2.db")

conn.execute("""
    CREATE TABLE categories 
    (
      id    INTEGER PRIMARY KEY, 
      name  TEXT NOT NULL
    );
""")

conn.execute(
    """
        INSERT INTO categories (name)
        VALUES
          ('casa'),
          ('laser'),
          ('faculdade')
    """
)
conn.commit()

In [106]:
conn.execute("""
    CREATE TABLE tasks 
    (
      id            INTEGER PRIMARY KEY AUTOINCREMENT, 
      description   TEXT NOT NULL,
      done          INTEGER,
      category_id   INTEGER NOT NULL,
      FOREIGN KEY (category_id) REFERENCES categories (id)
    )
""")
conn.commit()

In [108]:
# insert many
conn.execute(
    """
    INSERT INTO 'tasks' (description, done, category_id)
    VALUES
      ('Assistir Netflix', 1, 2),
      ('Estudar matplotlib', 0, 3),
      ('Estudar seaborn', 0, 3),
      ('Estudar Pandas', 0, 3),
      ('Lavar louça', 0, 1)
    """
)
conn.commit()

In [110]:
tasks_with_categories = conn.execute(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """
).fetchall()

for task in tasks_with_categories:
    print(task)

(5, 'Lavar louça', 0, 'casa')
(2, 'Estudar matplotlib', 0, 'faculdade')
(3, 'Estudar seaborn', 0, 'faculdade')
(4, 'Estudar Pandas', 0, 'faculdade')
(1, 'Assistir Netflix', 1, 'laser')


In [112]:
df = pd.read_sql_query(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """,
    conn
)
df


,id,description,done,name
0,5,Lavar louça,0,casa
1,2,Estudar matplotlib,0,faculdade
2,3,Estudar seaborn,0,faculdade
3,4,Estudar Pandas,0,faculdade
4,1,Assistir Netflix,1,laser


In [114]:
conn.close()